# Represent Protein Structure Data

在这部分，我们将学习如何获取一个基于结构的数据集进行预训练，并通过添加额外的边来增强每个样本，以更好地表示其结构。

## Protein Structure Dataset

首先，让我们构建一个蛋白质结构数据集。为了提高效率，我们基于datasets.EnzymeCommission定义了一个小型蛋白质结构数据集EnzymeCommissionToy。此外，我们将两个转换函数传递到数据集中，以截断过长的蛋白质并指定节点特征。

In [9]:
from torchdrug import datasets, transforms

# A toy protein structure dataset
class EnzymeCommissionToy(datasets.EnzymeCommission):
    url = "https://miladeepgraphlearningproteindata.s3.us-east-2.amazonaws.com/data/EnzymeCommission.tar.gz"
    md5 = "728e0625d1eb513fa9b7626e4d3bcf4d"
    processed_file = "enzyme_commission_toy.pkl.gz"
    test_cutoffs = [0.3, 0.4, 0.5, 0.7, 0.95]

truncuate_transform = transforms.TruncateProtein(max_length=350, random=False)
protein_view_transform = transforms.ProteinView(view='residue')
transform = transforms.Compose([truncuate_transform, protein_view_transform])

dataset = EnzymeCommissionToy("~/protein-datasets/", transform=transform, atom_feature=None, bond_feature=None)
train_set, valid_set, test_set = dataset.split()
print(dataset)
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))

19:25:38   Extracting /home/weibin/protein-datasets/EnzymeCommission.tar.gz to /home/weibin/protein-datasets


Loading /home/weibin/protein-datasets/EnzymeCommission/enzyme_commission_toy.pkl.gz: 100%|██████████| 1151/1151 [00:07<00:00, 145.12it/s]


EnzymeCommissionToy(
  #sample: 1151
  #task: 538
)
train samples: 959, valid samples: 97, test samples: 95


## Dynamic Graph Construction

从RDKit构建的蛋白质数据仅包含四种类型的键边（即单键，双键，三键或芳香键）。以数据集的第一个样本为例，我们挑选出前两个残基的原子，并展示它们之间的化学键。

In [10]:
from torchdrug import data

protein = dataset[0]["graph"]
is_first_two = (protein.residue_number == 1) | (protein.residue_number == 2)
first_two = protein.residue_mask(is_first_two, compact=True)
first_two.visualize()

为了更好地表示蛋白质结构，我们通过layers.GraphConstruction模块寻求动态重构蛋白质图。对于节点，我们使用layers.geometry.AlphaCarbonNode从蛋白质中提取Alpha碳来构建残基级别的图。对于边，我们使用layers.geometry.SpatialEdge、layers.geometry.KNNEdge和layers.geometry.SequentialEdge来构建不同残基之间的空间、KNN和顺序边（详细定义请参见 Tutorial 3 - Structure-based Protein Property Prediction.ipynb）。

In [11]:
from torchdrug import layers
from torchdrug.layers import geometry

graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()],
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)],
                                                    edge_feature="gearnet")

_protein = data.Protein.pack([protein])
protein_ = graph_construction_model(_protein)
print("Graph before: ", _protein)
print("Graph after: ", protein_)

degree = protein_.degree_in + protein_.degree_out
print("Average degree: ", degree.mean())
print("Maximum degree: ", degree.max())
print("Minimum degree: ", degree.min())

Graph before:  PackedProtein(batch_size=1, num_atoms=[2639], num_bonds=[5368], num_residues=[350])
Graph after:  PackedProtein(batch_size=1, num_atoms=[350], num_bonds=[7276], num_residues=[350])
Average degree:  tensor(41.5771)
Maximum degree:  tensor(76.)
Minimum degree:  tensor(12.)


经过这样的图构建，我们将蛋白质结构表示为一个残基级别的关系图。通过将空间边和KNN边视为两种类型的边，并将五个不同的顺序距离（即-2、-1、0、1和2）的顺序边视为五种边类型，我们得到一个具有7种不同边类型的关系图。每个边与一个59维边特征相关联，该特征是其两个端点节点的独热残基特征、边类型、顺序距离和空间距离的串联。

In [12]:
nodes_in, nodes_out, edges_type = protein_.edge_list.t()
residue_ids = protein_.residue_type.tolist()
for node_in, node_out, edge_type, edge_feature in zip(nodes_in.tolist()[:5], nodes_out.tolist()[:5], edges_type.tolist()[:5], protein_.edge_feature[:5]):
    print("[%s -> %s, type %d] edge feature shape: " % (data.Protein.id2residue[residue_ids[node_in]],
                                                        data.Protein.id2residue[residue_ids[node_out]], edge_type), edge_feature.shape)

[ILE -> VAL, type 1] edge feature shape:  torch.Size([59])
[TRP -> GLU, type 1] edge feature shape:  torch.Size([59])
[LEU -> GLU, type 1] edge feature shape:  torch.Size([59])
[VAL -> GLU, type 1] edge feature shape:  torch.Size([59])
[ARG -> ASP, type 1] edge feature shape:  torch.Size([59])


# Protein Structure Representation Model

TorchProtein定义了多种GNN模型，可以作为蛋白质结构编码器。在本教程中，我们采用了优秀的具有边消息传递的几何感知关系图神经网络（GearNet-Edge）。在TorchProtein中，我们可以使用models.GearNet定义一个GearNet-Edge模型。

In [13]:
from torchdrug import models

gearnet_edge = models.GearNet(input_dim=21, hidden_dims=[512, 512, 512],
                              num_relation=7, edge_input_dim=59, num_angle_bin=8,
                              batch_norm=True, concat_hidden=True, short_cut=True, readout="sum")

# Self-Supervised Protein Structure Pre-training

在这个教程中，我们采用了两种预训练算法，即**多视角对比学习**和**残基类型预测**，来从未标记的蛋白质结构中学习蛋白质表示。

## Multiview Contrastive Learning

Multiview对比学习旨在最大化同一蛋白质不同视图的表示之间的相似性，同时最小化不同蛋白质之间的相似性。下图说明了Multiview对比学习的高级思想。

![对比学习示意图](https://torchprotein.ai/assets/images/model/MultiviewContrast.png)

首先，我们将GearNet-Edge模型包装到models.MultiviewContrast模块中，其中我们通过aug_funcs参数传递数据增强函数，并通过crop_funcs参数传递裁剪函数。该模块在GearNet-Edge之上添加了一个MLP预测头。然后，将Multiview Contrast模块与图构建模型一起包装到tasks.Unsupervised模块中，用于自我监督的预训练。

在这里，我们使用了两种不同的裁剪函数：子序列（Subsequence）和子空间（Subspace）。前者会随机选择一个长度最多为50的连续子序列，而后者会选择以随机选定的中心残基为球心的球内的所有残基。在裁剪蛋白质之后，我们会随机选择是否随机遮盖残基图中的边作为一种数据增强。

In [14]:
from torchdrug import layers, models, tasks
from torchdrug.layers import geometry

model = models.MultiviewContrast(gearnet_edge, noise_funcs=[geometry.IdentityNode(), geometry.RandomEdgeMask(mask_rate=0.15)],
                                 crop_funcs=[geometry.SubsequenceNode(max_length=50),
                                             geometry.SubspaceNode(entity_level="residue", min_neighbor=15, min_radius=15.0)], num_mlp_layer=2)
task = tasks.Unsupervised(model, graph_construction_model=graph_construction_model)

现在我们可以开始预训练了。我们为模型设置一个优化器，并将所有内容放在一个Engine实例中。在这个预训练任务中，将模型训练10个epochs大约需要5分钟的时间。最后，我们将模型权重保存在最后一个epoch。

In [15]:
from torchdrug import core
import torch

optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=4)
solver.train(num_epoch=10)
solver.save("MultiviewContrast_ECToy.pth")

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## Residue Type Prediction

残基类型预测是一种典型的自我预测任务，它在输入的残基级图中屏蔽一部分残基，并试图根据蛋白质的上下文规律预测被屏蔽的残基类型。这种方法也被称为蛋白质的屏蔽逆折叠（根据结构预测序列）。下图说明了残基类型预测的高级思想。

![残基类型预测示意图](https://torchprotein.ai/assets/images/model/ResidueTypePrediction.png)


为了执行这个任务，我们将GearNet-Edge模型和图构建模型都包装到tasks.AttributeMasking模块中，在该模块中，将在GearNet-Edge之后添加一个MLP预测头。请注意，该模块也可以用于预训练分子编码器。该模块将根据训练集中图的视图选择是预测原子类型还是残基类型。

In [16]:
task = tasks.AttributeMasking(gearnet_edge, graph_construction_model=graph_construction_model,
                              mask_rate=0.15, num_mlp_layer=2)

现在我们可以开始预训练了。与上面类似，我们为模型设置一个优化器，并将所有内容放在一个Engine实例中。在这个预训练任务中，将模型训练10个epochs大约需要8分钟的时间。最后，我们将模型权重保存在最后一个epoch。

In [17]:
optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=4)
solver.train(num_epoch=10)
solver.save("ResidueTypePrediction_ECToy.pth")

19:25:55   Preprocess training set


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

# Fine-tuning on Downstream Task

我们使用小型酶委员会数据集上的蛋白质功能术语预测作为下游任务。这个任务旨在预测一个蛋白质是否具有几个特定功能，其中每个功能可以用二进制标签表示。因此，我们将这个任务形式化为多个二进制分类任务，并通过多任务学习的方式共同解决它们。我们使用tasks.MultipleBinaryClassification模块来执行这个任务，该模块将GearNet-Edge模型与一个MLP预测头结合起来。

In [18]:
task = tasks.MultipleBinaryClassification(gearnet_edge, graph_construction_model=graph_construction_model, num_mlp_layer=3,
                                          task=[_ for _ in range(len(dataset.tasks))], criterion="bce", metric=['auprc@micro', 'f1_max'])

## 1. Train from scratch

我们首先通过从头开始训练来评估GearNet-Edge模型。在这个任务上，将模型训练10个epochs大约需要8分钟的时间。最后，我们在验证集上进行评估。

In [19]:
optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=4)
solver.train(num_epoch=10)
solver.evaluate("valid")

19:25:57   Preprocess training set


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## 2. Fine-tune the Multiview Contrastive Learning model

接下来，我们评估通过多视角对比学习预训练的GearNet-Edge模型。我们使用预训练的模型权重初始化GearNet-Edge。在这个任务上，将模型训练10个epochs大约需要8分钟的时间。

In [20]:
optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=4)

_checkpoint = torch.load("MultiviewContrast_ECToy.pth")["model"]
checkpoint = {}
for k, v in _checkpoint.items():
    if k.startswith("model.model"):
        checkpoint[k[6:]] = v
    else:
        checkpoint[k] = v
checkpoint = {k: v for k, v in checkpoint.items() if not k.startswith("mlp")}
task.load_state_dict(checkpoint, strict=False)

solver.train(num_epoch=10)
solver.evaluate("valid")

19:26:24   Preprocess training set


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## 3. Fine-tune the Residue Type Prediction model

接下来，我们评估通过残基类型预测预训练的GearNet-Edge模型。在这个任务上，将模型训练10个epochs大约需要8分钟的时间。

In [ ]:
optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=4)

checkpoint = torch.load("ResidueTypePrediction_ECToy.pth")["model"]
checkpoint = {k: v for k, v in checkpoint.items() if not k.startswith("mlp")}
task.load_state_dict(checkpoint, strict=False)

solver.train(num_epoch=10)
solver.evaluate("valid")

注意: 我们观察到微调预训练模型的性能优于从头开始训练。然而，这两种方案的性能都不够令人满意，这主要归因于数据集大小过小。我们建议用户在更大的蛋白质结构数据集（例如datasets.AlphaFoldDB）上进行预训练，以充分调查预训练的有效性。